# Augmentation Library Imgaug 설치

In [1]:
!pip install imgaug

  Using cached imgaug-0.4.0-py2.py3-none-any.whl (948 kB)
  Using cached imageio-2.9.0-py3-none-any.whl (3.3 MB)
  Using cached scikit_image-0.17.2-cp37-cp37m-win_amd64.whl (11.5 MB)
  Using cached Shapely-1.7.1-cp37-cp37m-win_amd64.whl (1.0 MB)


# 라이브러리 불러오기

In [4]:
import glob
import imageio
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import xml.etree.ElementTree as ET
from PIL import Image
import os
%matplotlib inline

# 변수 세팅

In [5]:
label_info_path = 'classes.txt' # 텍스트 파일 만들고 label 을 순서대로 적어 놓자 
# aug folder path
aug_folder = 'data'
# img_file_path
img_file_path = aug_folder + "/*.jpg"
#xml_file_path 
xml_file_path = aug_folder + "/*.xml"
#txt_file_path
txt_file_path = aug_folder + "/*.txt"

#save_dir
save_dir = "aug"
#save_label
save_label = save_dir +"\\"+ "label"
#save_img
save_img = save_dir +"\\"+"img"

# Augmentation을 할 옵션들을 선택
augs = ["Add","Dropout","MotionBlur","AddtiveGaussianNoise","Sharpen","Emboss","Scale","Rotate","Shear","CropAndPad","WithBrightnessChannels","HorizontalFlip"]

#input format - xml, txt
input_format = 'xml'
#save format, yolo, xml 지금은 yolo만 구현
save_format = 'yolo'

#반복 횟수
iter_num = 3

## label mapping 중

In [6]:
labelmap ={}
labellist = []
with open(label_info_path, mode='r', encoding='utf-8') as f:
    label_list = f.readlines()
for i,label in enumerate(label_list):
    labelmap[label.replace('\n',"")] = i
    labellist.append(label.replace('\n',""))

# augmentation 할 파일 경로 리스트 세팅

In [7]:
## augmentation 할 파일 경로 리스트 세팅

img_path_list = glob.glob(img_file_path)
if input_format == 'xml':
    xml_path_list = glob.glob(xml_file_path)
elif input_format == 'txt':        
    txt_path_list = glob.glob(txt_file_path)

In [8]:
# img_path_list

In [9]:
# xml_path_list

In [10]:
# txt_path_list

# Augmentation 기능 매핑 및 추가

링크 타고 가서 어떤 효과인지는 찾아보세요 
https://imgaug.readthedocs.io/en/latest/source/overview/geometric.html


In [13]:
# aug 종류
#너무 많아서 손아파서 못쓰겠다 알아서 써라 


# augmentation 리스트 초기화
aug_list = []

if "Add" in augs:
    aug_list.append(iaa.Add((-40,40)))
if "AddElementwise" in augs:
    aug_list.append(iaa.AddElementwise((-40, 40)))
if "AddtiveGaussianNoise" in augs:
    aug_list.append(iaa.AdditiveGaussianNoise(scale=(0, 0.2*255), per_channel=True))
    aug_list.append(iaa.AdditiveGaussianNoise(scale=(0, 0.2*255)))
if "AddtiveLaplaceNoise" in augs:
    aug_list.append(iaa.AddtiveLaplaceNoise(scale=(0,0.2*255)))
    aug_list.append(iaa.AddtiveLaplaceNoise(scale=(0,0.2*255), per_channel=True))
if "AddtivePoissonNoise" in augs:
    aug_list.append(iaa.AdditivePoissonNoise(scale=(0, 40)))
if "Multiply" in augs:
    aug_list.append(iaa.Multiply((0.5, 1.5)))
if "MultiplyElementwise" in augs:
    aug_list.append(iaa.Cutout(nb_iterations=(1,5), fill_mode="gaussian", fill_per_channel=True))
    aug_list.append(iaa.Cutout(fill_mode="gaussian", fill_per_channel=False))
if "Dropout" in augs:
    aug_list.append(iaa.Dropout(p=(0,0.2)))
    aug_list.append(iaa.Dropout(p=(0,0.2), per_channel=0.5))
if "CoarseDropout" in augs:
    aug_list.append(iaa.CoarseDropout((0.0, 0.05), size_percent=(0.02,0.25)))
    aug_list.append(iaa.CoarseDropout(0.02, size_percent=0.15, per_channel=0.5))
if "ImpulseNoise" in augs:
    aug_list.append(iaa.ImpulseNoise(0.1))
if "SaltAndPepper" in augs:
    aug_list.append(iaa.SaltAndPepper(0.1), per_channel=True)
if "CoarseSaltAndPepper" in augs:
    aug_list.append(iaa.CoarseSaltAndPepper(
    0.05, size_percent=(0.01, 0.1), per_channel=True))
if "Invert" in augs:
    aug_list.append(iaa.Invert(0.25, per_channel=0.5))
if "Solarize" in augs:
    aug_list.append(iaa.Solarize(0.5, threshold=(32,128)))
if "JpegComppression" in augs:
    aug_list.append(iaa.JpegCompression(compression=(70,99)))
if "Cartoon" in augs:
    aug_list.append(iaa.Cartoon())
if "GaussianBlur" in augs:
    aug_list.append(iaa.GaussianBlur(sigma=(0.0,3.0)))
if "AverageBlur" in augs:
    aug_list.append(iaa.AverageBlur(k=(2,11)))
if "MedianBlur" in augs:
    aug_list.append(iaa.MedianBlur(k=(3,11)))
if "BilateralBlur" in augs:
    aug_list.append(iaa.BilateralBlur(
    d=(3, 10), sigma_color=(10, 250), sigma_space=(10, 250)))
if "MotionBlur" in augs:
    aug_list.append(iaa.MotionBlur(k=10,angle=[-45, 45]))
if "MeanShiftBlur" in augs:
    aug_list.append(iaa.MeanShiftBlur())
if "RandAugment" in augs: # 아무거나 적용하는 듯?
    aug_list.append(iaa.RandAugment(n=2,m=9))
    aug_list.append(iaa.RandAugment(m=30))
    aug_list.append(iaa.RandAugment(m=(0,9)))
if "WithColorspace" in augs:
    aug_list.append(iaa.WithColorspace(
    to_colorspace="HSV",
    from_colorspace="RGB",
    children=iaa.WithChannels(
        0,
        iaa.Add((0, 50))
    )
))
if "WithBrightnessChannels" in augs:
    aug_list.append(iaa.WithBrightnessChannels(iaa.Add((-50, 50))))
if "ChangeColorTemperature" in augs:
    aug_list.append(iaa.ChangeColorTemperature((1100, 10000)))
if "KMeansColorQuantization" in augs:
    aug_list.append(iaa.KMeansColorQuantization(n_colors=(4, 16)))
if "UniformColorQuantization" in augs:
    aug_list.append(iaa.UniformColorQuantization(n_colors=(4, 16)))
if "HorizontalFlip" in augs:
    aug_list.append(iaa.HorizontalFlip())
if "VerticalFlip" in augs:
    aug_list.append(iaa.VerticalFlip())
if "Fliplr" in augs:
    aug_list.append(iaa.Fliplr(0.5))
if "Fog" in augs:
    aug_list.append(iaa.imgcorruplike.Fog(severity=2))
if "Frost" in augs:
    aug_list.append(iaa.imgcorruplike.Frost(severity=2))
if "Snow" in augs:
    aug_list.append(iaa.imgcorruplike.Snow(severity=2))
if "Spatter" in augs:
    aug_list.append(iaa.imgcorruplike.Spatter(severity=2))
if "Pixelate" in augs:
    aug_list.append(iaa.imgcorruptlike.Pixelate(severity=2))
if "FastSnowyLandscape" in augs:
    aug_list.append(iaa.FastSnowyLandscape(
    lightness_threshold=[128, 200],
    lightness_multiplier=(1.5, 3.5)
))
if "Clouds" in augs:
    aug_list.append(iaa.Clouds())
if "Snowflakes" in augs:
    aug_list.append(flake_size=(0.2, 0.7), speed=(0.007, 0.03))
if "Rain" in augs:
    aug_list.append(iaa.Rain(speed=(0.1, 0.3)))
if "GammaContrast" in augs:
    aug_list.append(iaa.GammaContrast((0.5, 2.0)))
if "SigmoidContrast" in augs:
    aug_list.append(iaa.SigmoidContrast(gain=(3, 10), cutoff=(0.4, 0.6)))
if "LogContrast" in augs:
    aug_list.append(iaa.LogContrast(gain=(0.6,1.4)))
if "LinearContrast" in augs:
    aug_list.append(iaa.LinearContrast((0.4,1.6)))
if "CKAGE" in augs:
    aug_list.append(iaa.AllChannelsCLAHE(clip_limit=(1, 10)))
if "AllChannelsHistogramEqualization" in augs:
    aug_list.append(iaa.Alpha((0.0, 1.0), iaa.AllChannelsHistogramEqualization()))
if "Convolve" in augs:
    matrix = np.array([[0, -1, 0],
                   [-1, 4, -1],
                   [0, -1, 0]])
    aug_list.append(iaa.Convolve(matrix=matrix))
if "Sharpen" in augs:
    aug_list.append(iaa.Sharpen(alpha=(0.0, 1.0), lightness=(0.75, 2.0)))
if "Emboss" in augs:
    aug_list.append(iaa.Emboss(alpha=(0.0,1.0), strength=(0.5,1.5)))
if "Scale" in augs:
    aug_list.append(iaa.Affine(scale=(0.5, 1.5)))
if "Rotate" in augs:
    aug_list.append(iaa.Affine(rotate=(-45,45)))
if "Shear" in augs:
    aug_list.append(iaa.Affine(shear=(-16,16)))
if "PiecewiseAffine" in augs:
    aug_list.append( iaa.PiecewiseAffine(scale=(0.01, 0.05)))
if "PerspectiveTransform" in augs:
    aug_list.append(iaa.PerspectiveTransform(scale=(0.01, 0.15)))
if "ElasticTransformation" in augs:
    aug_list.append(iaa.ElasticTransformation(alpha=(0, 5.0), sigma=0.25))
if "CropAndPad" in augs:
    aug_list.append(iaa.CropAndPad(
        percent=(-0.05, 0.1),
            pad_mode=ia.ALL,
            pad_cval=(0, 255)
    ))

In [14]:
augs

['Add',
 'Dropout',
 'MotionBlur',
 'AddtiveGaussianNoise',
 'Sharpen',
 'Emboss',
 'Scale',
 'Rotate',
 'Shear',
 'CropAndPad',
 'WithBrightnessChannels',
 'HorizontalFlip']

# Augmentation 객체 생성

In [15]:
seq = iaa.Sequential( ## 선택된 augmentator 중에서 0~전체 개수의 augmentator에서 랜덤한 순서대로 선택하여 진행
    iaa.SomeOf((0, None),
              aug_list,
              random_order=True)
)
seq2 = iaa.Sequential( ## 하나만 랜덤하게 골라서 진행
    iaa.OneOf(aug_list)
)

# VOC 형식의 XML 데이터에서 바운딩 박스와 라벨 데이터 추출하는 함수

In [16]:
def getXMLDATA(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot() # 루트 파싱
    file_info ={}
    objects_list = []
    x = file_path.split('\\')
    x.reverse()
    #파일 명 추가
    file_info['filename'] = x[0]
    #사진 크기 추가
    file_info['width'] = root.find("size").find("width").text
    file_info['height'] = root.find("size").find("height").text
    #object 리스트 추가 
    for o in root.iter('object'):
        bndbox = {}
        bndbox['label'] = o.find("name").text # 라벨링 추가 
        sizes = []
        sizes.append(o.find("bndbox").find("xmin").text)
        sizes.append(o.find("bndbox").find("ymin").text)
        sizes.append(o.find("bndbox").find("xmax").text)
        sizes.append(o.find("bndbox").find("ymax").text)
        bndbox["bndbox"] = sizes
        objects_list.append(bndbox)
    file_info['objects'] = objects_list
    return file_info

# yolov Darknet 의 txt 데이터에서 바운딩 박스와 라벨 데이터 추출

In [17]:
def getImgSize(img_path):
    image = Image.open(img_path)
    width, height = image.size
    size =[]
    size.append(width)
    size.append(height)
    return size

def getYoloDATA(file_path, img_size):
    with open(file_path, mode='r', encoding='utf-8') as f:
        datas=f.readlines()
    file_info = {}
    x = file_path.split('\\')
    x.reverse()
    #파일 명 추가
    file_info['filename'] = x[0]
    #사진 크기 추가
    file_info['width'] = img_size[0]
    file_info['height'] = img_size[1]
    #object 리스트 추가
    objects_list = []
    for i in datas:
        i = i.replace("\n","")
        bndbox = {}
        temp = i.split(' ')
        bndbox['label'] = temp[0]
        sizes = []
        xmax = (float(temp[1])*img_size[0]*2 + float(temp[3])*img_size[0])/2.0
        xmin = (float(temp[1])*img_size[0]*2 - float(temp[3])*img_size[0])/2.0
        ymax = (float(temp[2])*img_size[1]*2 + float(temp[4])*img_size[1])/2.0
        ymin = (float(temp[2])*img_size[1]*2 - float(temp[4])*img_size[1])/2.0
        sizes.append(xmin)
        sizes.append(ymin)
        sizes.append(xmax)
        sizes.append(ymax)
        bndbox['bndbox'] = sizes
        objects_list.append(bndbox)
    file_info['objects'] = objects_list
    return file_info
        
#         dw = 1./416
#         dh = 1./416
#         x = ((int(box[0][0])+int(box[1][0]))/2.0)*dw
#         y = ((int(box[0][1])+int(box[1][1]))/2.0)*dh
#         w = (int(box[1][0]) - int(box[0][0]))*dw
#         h = (int(box[1][1]) - int(box[0][1]))*dh     
    

# XML file Augmentation 진행 ->save yolo

In [19]:
if input_format == 'xml':
    for k in range(iter_num):
        for i, path in enumerate(xml_path_list):
            data = getXMLDATA(path)
            #확장자 버리고 이름만 따기
            name = data['filename'].split('.')
            name = name[0]
            #이미지 읽어오기
            image = imageio.imread(img_path_list[i])
            #이미지 416 으로 리사이즈
            image = ia.imresize_single_image(image,(416,416))

            #resize 이후 라벨링 비율을 바꾸기 위함
            x_ratio = 416/int(data["width"])
            y_ratio = 415/int(data["height"])

            # 라벨 데이터와 바운딩 박스를 분리
            objects = data["objects"]

            text_list = []
            bbs_list = []

            for obj in objects:
                text_list.append(str(labelmap[obj['label']])) #라벨들을 미리 숫자로 바꾸고 text 앞부분에 저장        
                bbs = BoundingBox(x1=round(int(obj['bndbox'][0])*x_ratio),
                                  y1=round(int(obj['bndbox'][1])*y_ratio),
                                  x2=round(int(obj['bndbox'][2])*x_ratio),
                                  y2=round(int(obj['bndbox'][3])*y_ratio))
                bbs_list.append(bbs)
        #         ia.imshow(bbs.draw_on_image(image, size=2)) # 이미지 출력해서 원본 이미지 라벨링 보고 싶다면 ㄱ
            #바운딩 박스 객체 생성
            bounding_box = BoundingBoxesOnImage(bbs_list, shape=image.shape)
            # augmentation 진행
            img_aug, bbs_aug = seq(image=image, bounding_boxes=bounding_box)
            print(bbs_aug[0])

        #     ia.imshow(bbs_aug.draw_on_image(img_aug, size=2)) # 변환된 이미지 보고 싶다면 ㄱ

            # numpy 형식의 데이터인 이미지를 저장
            try: # 폴더 경로가 없는 경우 생성
                if not(os.path.isdir(save_img)):
                    os.makedirs(os.path.join(save_img))
                if not(os.path.isdir(save_label)):
                    os.makedirs(os.path.join(save_label))
            except OSError as e:
                if e.errno != errno.EXIST:
                    print("Failed to create directory!!!!!")
                    raise


            if save_format == "yolo":
                im = Image.fromarray(img_aug)
                im.save(save_img+"\\"+name+'_aug' + str(k)+ '.jpg')


                for i,box in enumerate(bbs_aug):
                    dw = 1./416
                    dh = 1./416
                    x = ((int(box[0][0])+int(box[1][0]))/2.0)*dw
                    y = ((int(box[0][1])+int(box[1][1]))/2.0)*dh
                    w = (int(box[1][0]) - int(box[0][0]))*dw
                    h = (int(box[1][1]) - int(box[0][1]))*dh
                    text_list[i] += ' ' + str(x) +' '+ str(y)  +' ' + str(w) +' ' +str(h)

                for i in text_list:
                    with open(save_label+"\\"+name+'_aug' + str(k)+ '.txt', mode='wt', encoding='utf-8') as f:
                        f.write(i+'\n')

with open(save_img+'\\'+"classes.labels", mode='wt', encoding='utf-8') as f:
    for i in labellist:
        f.write(i+'\n')

BoundingBox(x1=185.3007, y1=196.9126, x2=207.5367, y2=208.4435, label=None)
BoundingBox(x1=251.2893, y1=202.0309, x2=279.7969, y2=210.2958, label=None)
BoundingBox(x1=178.0000, y1=188.0000, x2=196.0000, y2=212.0000, label=None)
BoundingBox(x1=198.0000, y1=321.0000, x2=207.0000, y2=359.0000, label=None)
BoundingBox(x1=208.6613, y1=220.9195, x2=223.5628, y2=244.7647, label=None)
BoundingBox(x1=156.9936, y1=268.5011, x2=178.2755, y2=300.7521, label=None)
BoundingBox(x1=176.9179, y1=269.6587, x2=212.2050, y2=327.4572, label=None)
BoundingBox(x1=290.5030, y1=210.3799, x2=307.8661, y2=232.9797, label=None)
BoundingBox(x1=78.4943, y1=55.3991, x2=276.5369, y2=281.2655, label=None)
BoundingBox(x1=-27.7742, y1=-24.5747, x2=457.4033, y2=453.2568, label=None)
BoundingBox(x1=181.7167, y1=177.1619, x2=387.8649, y2=410.7209, label=None)
BoundingBox(x1=0.4414, y1=77.8314, x2=422.3097, y2=478.3620, label=None)
BoundingBox(x1=30.1152, y1=306.8943, x2=439.3066, y2=412.4445, label=None)
BoundingBox(x1=1.0

BoundingBox(x1=39.7936, y1=80.6794, x2=349.1250, y2=343.6820, label=None)
BoundingBox(x1=173.2875, y1=26.8598, x2=300.4781, y2=333.8008, label=None)
BoundingBox(x1=1.0000, y1=202.0000, x2=336.0000, y2=413.0000, label=None)
BoundingBox(x1=110.6429, y1=219.7931, x2=445.8883, y2=436.0959, label=None)
BoundingBox(x1=-22.9705, y1=-30.4083, x2=456.7694, y2=460.2317, label=None)
BoundingBox(x1=132.0635, y1=273.3185, x2=257.5238, y2=297.4076, label=None)
BoundingBox(x1=-92.1057, y1=26.1223, x2=351.7900, y2=434.1898, label=None)
BoundingBox(x1=290.8662, y1=113.3525, x2=462.7554, y2=262.7731, label=None)
BoundingBox(x1=-75.1367, y1=-96.8718, x2=486.7544, y2=479.4116, label=None)
BoundingBox(x1=64.8388, y1=148.0685, x2=159.0809, y2=193.5590, label=None)
BoundingBox(x1=-36.7131, y1=26.5153, x2=457.6076, y2=521.8050, label=None)
BoundingBox(x1=-177.0887, y1=-192.4094, x2=605.8998, y2=591.8488, label=None)
BoundingBox(x1=94.6866, y1=224.4717, x2=332.4052, y2=374.8671, label=None)
BoundingBox(x1=188.

BoundingBox(x1=295.1798, y1=260.5626, x2=360.0541, y2=339.2809, label=None)
BoundingBox(x1=-227.5259, y1=-306.2697, x2=561.5336, y2=626.4849, label=None)
BoundingBox(x1=18.5895, y1=93.5644, x2=402.4162, y2=365.4453, label=None)
BoundingBox(x1=-5.0268, y1=59.9133, x2=395.3603, y2=353.7749, label=None)
BoundingBox(x1=24.8073, y1=109.5758, x2=340.6239, y2=227.8788, label=None)
BoundingBox(x1=89.5962, y1=5.9022, x2=486.2556, y2=423.7320, label=None)
BoundingBox(x1=-75.1825, y1=-75.7038, x2=448.2469, y2=419.4989, label=None)
BoundingBox(x1=65.7263, y1=131.9839, x2=263.0227, y2=299.0914, label=None)
BoundingBox(x1=21.0000, y1=105.0000, x2=337.0000, y2=365.0000, label=None)
BoundingBox(x1=26.5897, y1=34.0819, x2=379.9753, y2=338.8145, label=None)
BoundingBox(x1=70.3692, y1=114.8049, x2=409.4786, y2=341.0016, label=None)
BoundingBox(x1=-17.3749, y1=-32.6741, x2=337.3710, y2=293.2816, label=None)
BoundingBox(x1=-25.4707, y1=40.2285, x2=402.3400, y2=307.8777, label=None)
BoundingBox(x1=29.0000, 

BoundingBox(x1=86.2575, y1=38.4559, x2=426.6857, y2=350.2350, label=None)
BoundingBox(x1=39.2139, y1=44.9628, x2=343.3441, y2=329.9904, label=None)
BoundingBox(x1=97.6316, y1=68.2000, x2=439.0379, y2=346.3284, label=None)
BoundingBox(x1=143.8590, y1=89.8454, x2=325.1564, y2=263.5313, label=None)
BoundingBox(x1=-54.6694, y1=-32.7665, x2=450.4803, y2=241.8950, label=None)
BoundingBox(x1=-5.0006, y1=204.1523, x2=412.4060, y2=385.6467, label=None)
BoundingBox(x1=87.2788, y1=-30.3376, x2=275.7798, y2=340.6436, label=None)
BoundingBox(x1=136.0000, y1=125.0000, x2=347.0000, y2=219.0000, label=None)
BoundingBox(x1=-19.1012, y1=-23.5695, x2=446.0550, y2=437.8562, label=None)
BoundingBox(x1=155.3331, y1=88.2147, x2=423.7452, y2=335.1087, label=None)
BoundingBox(x1=45.0041, y1=75.1020, x2=373.8601, y2=383.1271, label=None)
BoundingBox(x1=-65.0465, y1=53.8463, x2=536.2557, y2=535.0352, label=None)
BoundingBox(x1=23.0000, y1=1.0000, x2=416.0000, y2=413.0000, label=None)
BoundingBox(x1=69.2121, y1=7

BoundingBox(x1=-99.8176, y1=-92.3963, x2=519.8878, y2=523.7010, label=None)
BoundingBox(x1=19.4663, y1=-10.3582, x2=378.4453, y2=361.6978, label=None)
BoundingBox(x1=-2.3570, y1=64.1367, x2=450.0220, y2=453.1319, label=None)
BoundingBox(x1=164.0035, y1=198.8452, x2=244.3144, y2=287.9363, label=None)
BoundingBox(x1=-40.5701, y1=-39.3751, x2=456.5701, y2=396.8177, label=None)
BoundingBox(x1=135.0000, y1=250.0000, x2=357.0000, y2=415.0000, label=None)
BoundingBox(x1=-9.5304, y1=19.7278, x2=431.9491, y2=395.5349, label=None)
BoundingBox(x1=-43.6079, y1=-42.8182, x2=435.7961, y2=426.8406, label=None)
BoundingBox(x1=76.9169, y1=48.4454, x2=483.3099, y2=433.9079, label=None)
BoundingBox(x1=-129.2437, y1=96.3491, x2=393.4275, y2=453.7280, label=None)
BoundingBox(x1=-26.6387, y1=48.7945, x2=416.5318, y2=429.9906, label=None)
BoundingBox(x1=58.0186, y1=71.7789, x2=409.1659, y2=404.4718, label=None)
BoundingBox(x1=-0.1333, y1=-34.9638, x2=383.1587, y2=355.5563, label=None)
BoundingBox(x1=33.2441,

...

BoundingBox(x1=97.4416, y1=191.6005, x2=305.8491, y2=230.0993, label=None)
BoundingBox(x1=1.4940, y1=69.6742, x2=285.9247, y2=377.1888, label=None)
BoundingBox(x1=-57.5178, y1=-52.3870, x2=474.8005, y2=361.9233, label=None)
BoundingBox(x1=-17.2219, y1=-45.2187, x2=432.4188, y2=416.5690, label=None)
BoundingBox(x1=86.4987, y1=103.7377, x2=345.5172, y2=318.2046, label=None)
BoundingBox(x1=-88.1671, y1=-68.0392, x2=473.9753, y2=445.2212, label=None)
BoundingBox(x1=-4.9751, y1=-39.4241, x2=456.8593, y2=414.1285, label=None)
BoundingBox(x1=159.0000, y1=87.0000, x2=223.0000, y2=249.0000, label=None)
BoundingBox(x1=62.3918, y1=17.0311, x2=277.0112, y2=203.2868, label=None)
BoundingBox(x1=25.0000, y1=54.0000, x2=397.0000, y2=393.0000, label=None)
BoundingBox(x1=83.5108, y1=120.1840, x2=321.4309, y2=314.1367, label=None)
BoundingBox(x1=0.0000, y1=1.0000, x2=412.0000, y2=335.0000, label=None)
BoundingBox(x1=33.9723, y1=-20.2962, x2=393.9680, y2=419.7540, label=None)
BoundingBox(x1=-17.2085, y1=-

BoundingBox(x1=5.0000, y1=147.0000, x2=13.0000, y2=157.0000, label=None)
BoundingBox(x1=192.1686, y1=166.0475, x2=208.5471, y2=181.1976, label=None)
BoundingBox(x1=194.2642, y1=239.8638, x2=203.0943, y2=243.4043, label=None)
BoundingBox(x1=142.7350, y1=87.3532, x2=266.7961, y2=231.9640, label=None)
BoundingBox(x1=180.7063, y1=160.3229, x2=210.6799, y2=231.2258, label=None)
BoundingBox(x1=178.4780, y1=196.6659, x2=209.1840, y2=256.3971, label=None)
BoundingBox(x1=210.8549, y1=176.4823, x2=235.6046, y2=228.6003, label=None)
BoundingBox(x1=182.8626, y1=175.2029, x2=222.8030, y2=216.6191, label=None)
BoundingBox(x1=191.5168, y1=166.6904, x2=207.7040, y2=207.2604, label=None)
BoundingBox(x1=178.1258, y1=130.6086, x2=225.0243, y2=210.4892, label=None)
BoundingBox(x1=175.1268, y1=167.5715, x2=216.8837, y2=217.4281, label=None)
BoundingBox(x1=174.4812, y1=160.5614, x2=190.0927, y2=206.1754, label=None)
BoundingBox(x1=190.9308, y1=213.1714, x2=201.2386, y2=225.8705, label=None)
BoundingBox(x1=1

BoundingBox(x1=178.9293, y1=225.8176, x2=198.6223, y2=255.8260, label=None)
BoundingBox(x1=178.0000, y1=225.0000, x2=198.0000, y2=258.0000, label=None)
BoundingBox(x1=221.7218, y1=219.4970, x2=257.5259, y2=264.7850, label=None)
BoundingBox(x1=228.8890, y1=207.6601, x2=257.7474, y2=237.8102, label=None)
BoundingBox(x1=216.2085, y1=201.1076, x2=235.4396, y2=219.7440, label=None)
BoundingBox(x1=188.6466, y1=243.0834, x2=223.8655, y2=284.8669, label=None)
BoundingBox(x1=171.7678, y1=215.9726, x2=184.9119, y2=235.2826, label=None)
BoundingBox(x1=183.0000, y1=221.0000, x2=202.0000, y2=248.0000, label=None)
BoundingBox(x1=186.6008, y1=221.0782, x2=207.5221, y2=246.8004, label=None)
BoundingBox(x1=178.8259, y1=217.4229, x2=199.4732, y2=241.2462, label=None)
BoundingBox(x1=185.0000, y1=217.0000, x2=202.0000, y2=236.0000, label=None)
BoundingBox(x1=187.0000, y1=215.0000, x2=201.0000, y2=236.0000, label=None)
BoundingBox(x1=213.3673, y1=223.0572, x2=243.9858, y2=259.5272, label=None)
BoundingBox(

BoundingBox(x1=188.5488, y1=244.5239, x2=208.4838, y2=274.5072, label=None)
BoundingBox(x1=318.1764, y1=273.9579, x2=383.4419, y2=346.2911, label=None)
BoundingBox(x1=177.3645, y1=311.3099, x2=212.6649, y2=368.6135, label=None)
BoundingBox(x1=199.9246, y1=239.5979, x2=270.3426, y2=320.8773, label=None)
BoundingBox(x1=119.6630, y1=282.1973, x2=131.9931, y2=299.2442, label=None)
BoundingBox(x1=165.3391, y1=304.5078, x2=177.9583, y2=329.2434, label=None)
BoundingBox(x1=138.0000, y1=318.0000, x2=149.0000, y2=339.0000, label=None)
BoundingBox(x1=292.9389, y1=276.0130, x2=313.7641, y2=301.0181, label=None)
BoundingBox(x1=187.0946, y1=315.4217, x2=210.6728, y2=343.1452, label=None)
BoundingBox(x1=167.0549, y1=351.6487, x2=191.2077, y2=385.9063, label=None)
BoundingBox(x1=163.3599, y1=310.0995, x2=174.3094, y2=329.4632, label=None)
BoundingBox(x1=265.8809, y1=308.0298, x2=275.8229, y2=325.2096, label=None)
BoundingBox(x1=264.0342, y1=255.8219, x2=279.8531, y2=275.1271, label=None)
BoundingBox(

IndexError: list index out of range

# Yolo augmentation ->yolo save

In [ ]:
if input_format == 'txt':
    for k in range(iter_num):
        for i, path in enumerate(txt_path_list):
            data = getYoloDATA(path,getImgSize(img_path_list[i]))
            #확장자 버리고 이름만 따기
            name = data['filename'].split('.')
            name = name[0]
            #이미지 읽어오기
            image = imageio.imread(img_path_list[i])
            #이미지 416 으로 리사이즈
            image = ia.imresize_single_image(image,(416,416))


            #resize 이후 라벨링 비율을 바꾸기 위함
            x_ratio = 416/int(data["width"])
            y_ratio = 416/int(data["height"])

            # 라벨 데이터와 바운딩 박스를 분리
            objects = data["objects"]

            text_list = []
            bbs_list = []

            for obj in objects:
                text_list.append(str(obj['label'])) #라벨들을 미리 숫자로 바꾸고 text 앞부분에 저장        
                bbs = BoundingBox(x1=round(int(obj['bndbox'][0])*x_ratio),
                                  y1=round(int(obj['bndbox'][1])*y_ratio),
                                  x2=round(int(obj['bndbox'][2])*x_ratio),
                                  y2=round(int(obj['bndbox'][3])*y_ratio))
                bbs_list.append(bbs)
        #         ia.imshow(bbs.draw_on_image(image, size=2)) # 이미지 출력해서 원본 이미지 라벨링 보고 싶다면 ㄱ
            #바운딩 박스 객체 생성
            bounding_box = BoundingBoxesOnImage(bbs_list, shape=image.shape)
            # augmentation 진행
            img_aug, bbs_aug = seq(image=image, bounding_boxes=bounding_box)


        #     ia.imshow(bbs_aug.draw_on_image(img_aug, size=2)) # 변환된 이미지 보고 싶다면 ㄱ

            # numpy 형식의 데이터인 이미지를 저장
            try: # 폴더 경로가 없는 경우 생성
                if not(os.path.isdir(save_img)):
                    os.makedirs(os.path.join(save_img))
                if not(os.path.isdir(save_label)):
                    os.makedirs(os.path.join(save_label))
            except OSError as e:
                if e.errno != errno.EXIST:
                    print("Failed to create directory!!!!!")
                    raise


            if save_format == "yolo":
                im = Image.fromarray(img_aug)
                im.save(save_img+"\\"+name+'_aug' + str(k)+ '.jpg')

                for i,box in enumerate(bbs_aug):
                    dw = 1./416
                    dh = 1./416
                    x = ((int(box[0][0])+int(box[1][0]))/2.0)*dw
                    y = ((int(box[0][1])+int(box[1][1]))/2.0)*dh
                    w = (int(box[1][0]) - int(box[0][0]))*dw
                    h = (int(box[1][1]) - int(box[0][1]))*dh
                    text_list[i] += ' ' + str(x) +' '+ str(y)  +' ' + str(w) +' ' +str(h)

                for i in text_list:
                    with open(save_label+"\\"+name+'_aug' + str(k)+ '.jpg', mode='wt', encoding='utf-8') as f:
                        f.write(i+'\n')

with open(save_img+'\\'+"classes.labels", mode='wt', encoding='utf-8') as f:
    for i in labellist:
        f.write(i+'\n')